In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
from seminartools.models.distance_model import DistanceModel
from seminartools.models.random_effects_model import RandomEffectsModel
from seminartools.models.uc_sv_ss_model import UCSVSSModel
from seminartools.data import read_merged
import pandas as pd

In [85]:
df = read_merged().reset_index()
df

c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:161: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").apply(lambda x: (1 + x).prod() - 1)  # quarterly return
c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:397: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").mean()  # Mean interest rate


,country,date,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
0,Japan,1977-01-01,0.022951,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
1,Canada,1977-01-01,0.023904,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
2,United States,1977-01-01,0.017231,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
3,Japan,1977-04-01,0.027244,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
4,Canada,1977-04-01,0.023346,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...,...
2166,Italy,2023-01-01,0.003938,0.552944,3.000000,8.3,-0.02041,-0.585458,0.002037,0.007418,0.043183
2167,Canada,2023-01-01,0.006080,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
2168,Sweden,2023-01-01,0.014955,0.656897,2.833333,7.8,-0.02041,-0.585458,0.002037,0.007418,0.043183
2169,Korea,2023-01-01,0.010538,0.329875,3.500000,3.2,-0.02041,-0.585458,0.002037,0.007418,0.043183


In [86]:
distance_model = DistanceModel()
distance_model

In [87]:
distance_model.fit(df)

Creating regression coefficients: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]


In [88]:
preds_dist = distance_model.predict(df, aggregation_method="distribution")

In [89]:
from scipy.integrate import simps
australia = preds_dist.iloc[0]
print(simps(australia.inflation["pdf"]))

0.9999999999999999


C:\Users\vandi\AppData\Local\Temp\ipykernel_9412\980366153.py:3: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  print(simps(australia.inflation["pdf"]))


In [90]:
from seminartools.model_evaluation import get_95_interval
interval = get_95_interval(australia.inflation["pdf"], australia.inflation["inflation_grid"])
lower_bound = interval[0]
upper_bound = interval[1]

lower_bound, upper_bound


(0.014001062138216466, 0.01692830934753002)

In [97]:
from seminartools.models.utils import make_oos_predictions
from seminartools.time_series_split import ExpandingWindowSplit

model = DistanceModel()

tss = ExpandingWindowSplit(start_date = "2000-01-01")
preds = make_oos_predictions(lambda: DistanceModel(), df, tss, progress = True, num_cores = 5, density = True)


Splits: 100%|██████████| 5/5 [00:00<00:00, 148.38it/s]


TypeError: DistanceModel.fit() got an unexpected keyword argument 'pointwise_aggregation_method'

In [82]:
in_interval = np.logical_and(inflation >= lower_bound, inflation <= upper_bound)


NameError: name 'np' is not defined